<a href="https://colab.research.google.com/github/hamzaq453/Langchain-learnings/blob/main/Agent_with_Memory_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!pip install -q -U langchain_google_genai langchain_core langgraph


In [36]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.prebuilt import tools_condition, ToolNode
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver
import IPython.display as display

# Load the GEMINI API key
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

# Initialize the LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=GEMINI_API_KEY)

# Define Tools
def add(a: int, b: int) -> int:
    """Adds two numbers."""
    return a + b

def multiply(a: int, b: int) -> int:
    """Multiplies two numbers."""
    return a * b

def divide(a: int, b: int) -> float:
    """Divides two numbers."""
    if b == 0:
        raise ValueError("Division by zero is not allowed.")
    return a / b

# Bind tools to the LLM
tools = [add, multiply, divide]
llm_with_tools = llm.bind_tools(tools)

# Define system message
sys_msg = SystemMessage(content="You are a helpful assistant that performs arithmetic operations.")

# Assistant node
def assistant(state: MessagesState) -> MessagesState:
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

# Initialize memory checkpointer
memory = MemorySaver()

# Build the graph
builder = StateGraph(MessagesState)

# Add nodes
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    tools_condition,  # Route to tools if a tool call is detected
)
builder.add_edge("tools", "assistant")  # Loop back to assistant
builder.add_edge("assistant", END)  # End if no tool call is detected

# Compile the graph with memory
react_graph_memory = builder.compile(checkpointer=memory)

# Interactive Chatbot Function
def chat_with_memory():
    print("Chatbot: Hello! I can perform arithmetic operations for you. Type 'quit' or 'exit' to end the chat.")
    thread_id = "user_session_1"  # Use a static thread ID for persistent memory
    config = {"configurable": {"thread_id": thread_id}}

    while True:
        # Take user input
        user_input = input("You: ")

        # Exit condition
        if user_input.lower() in ["quit", "exit"]:
            print("Chatbot: Goodbye! Have a nice day.")
            break

        # Prepare user message
        messages = [HumanMessage(content=user_input)]

        # Invoke the graph
        response = react_graph_memory.invoke({"messages": messages}, config)

        # Display the AI's response
        for msg in response['messages']:
            if hasattr(msg, 'pretty_print'):
                msg.pretty_print()
            else:
                print(msg)

# Start the chatbot
chat_with_memory()


Chatbot: Hello! I can perform arithmetic operations for you. Type 'quit' or 'exit' to end the chat.
You: Hi
================================ Human Message =================================

Hi
================================== Ai Message ==================================

Hello! How can I help you today?
You: What is 2 multiply 4
================================ Human Message =================================

Hi
================================== Ai Message ==================================

Hello! How can I help you today?
================================ Human Message =================================

What is 2 multiply 4
================================== Ai Message ==================================
Tool Calls:
  multiply (0d15c1d3-e019-4638-a334-de32bb712b3b)
 Call ID: 0d15c1d3-e019-4638-a334-de32bb712b3b
  Args:
    a: 2.0
    b: 4.0
================================= Tool Message =================================
Name: multiply

8
================================== Ai Messag